In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import re
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, average_precision_score
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from sklearn.model_selection import StratifiedKFold
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import ShuffleSplit

In [2]:
train_data = pd.read_csv('./train_dataset/train_public.csv')
test_data = pd.read_csv('./test_public.csv')
sub=pd.read_csv("./submit/test1.csv")
sub=sub.rename(columns={'id': 'loan_id'})
sub.loc[sub['isDefault']<0.5,'isDefault'] = 0
nw_sub=sub[(sub['isDefault']==0)]
nw_test_data=test_data.merge(nw_sub,on='loan_id',how='inner')
nw_train_data = pd.concat([train_data,nw_test_data]).reset_index(drop=True)
nw_train_data.to_csv("./train_dataset/nw_train_public.csv",index=0)

In [6]:
nw_test_data

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,work_year,...,policy_code,f0,f1,f2,f3,f4,early_return,early_return_amount,early_return_amount_3mon,isDefault
0,1000575,200575,2890.909091,3,10.791,88.01,B,幼教与中小学校,住宿和餐饮业,5 years,...,1,2.0,0.0,15.0,5.0,4.0,3,773,89.192308,0.0
1,1028125,228125,7272.727273,3,9.990,258.10,B,普通企业,批发和零售业,10+ years,...,1,8.0,0.0,8.0,29.0,14.0,1,1894,218.538462,0.0
2,1010694,210694,26295.454550,3,15.763,764.03,C,普通企业,住宿和餐饮业,10+ years,...,1,6.0,0.0,4.0,10.0,6.0,1,5670,1221.230769,0.0
3,1026712,226712,22690.909090,5,19.305,524.30,D,普通企业,采矿业,10+ years,...,1,4.0,0.0,12.0,10.0,8.0,2,4800,443.076923,0.0
4,1002895,202895,14545.454550,3,7.139,490.32,A,世界五百强,金融业,1 year,...,1,4.0,0.0,7.0,14.0,9.0,0,3516,649.107692,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4600,1008856,208856,9454.545455,5,12.015,183.47,C,政府机构,信息传输、软件和信息技术服务业,< 1 year,...,1,NaN,NaN,NaN,NaN,NaN,0,0,0.000000,0.0
4601,1016651,216651,5500.000000,3,7.970,172.28,A,政府机构,房地产业,5 years,...,1,3.0,0.0,2.0,5.0,3.0,3,1564,0.000000,0.0
4602,1024140,224140,30545.454550,3,8.900,889.09,A,上市企业,房地产业,10+ years,...,1,8.0,0.0,17.0,20.0,14.0,2,5456,1510.892308,0.0
4603,1014316,214316,4090.909091,3,6.030,152.18,A,政府机构,文化和体育业,10+ years,...,1,1.0,0.0,6.0,10.0,10.0,3,223,41.169231,0.0


In [3]:
train_data = pd.read_csv('./train_dataset/nw_train_public.csv')
submit_example = pd.read_csv('./submit_example.csv')
test_public = pd.read_csv('./test_public.csv')
train_internet = pd.read_csv('./train_dataset/train_internet.csv', encoding='gb2312')

## 模型 

In [4]:
def train_model(data_, test_, y_, folds_):
    oof_preds = np.zeros(data_.shape[0])
    sub_preds = np.zeros(test_.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in data_.columns if f not in ['loan_id', 'user_id', 'isDefault'] ]
    for n_fold, (trn_idx, val_idx) in enumerate(folds_.split(data_)):
        trn_x, trn_y = data_[feats].iloc[trn_idx], y_.iloc[trn_idx]
        val_x, val_y = data_[feats].iloc[val_idx], y_.iloc[val_idx]
        clf = LGBMClassifier(
            n_estimators=4000,
            learning_rate=0.08,
            num_leaves=2**5,
            colsample_bytree=.65,
            subsample=.9,
            max_depth=5,
#             max_bin=250,
            reg_alpha=.3,
            reg_lambda=.3,
            min_split_gain=.01,
            min_child_weight=2,
            silent=-1,
            verbose=-1,
        )
        
        clf.fit(trn_x, trn_y, 
                eval_set= [(trn_x, trn_y), (val_x, val_y)], 
                eval_metric='auc', verbose=100, early_stopping_rounds=50  #30
               )

        oof_preds[val_idx] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_[feats], num_iteration=clf.best_iteration_)[:, 1] / folds_.n_splits
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(val_y, oof_preds[val_idx])))
        del clf, trn_x, trn_y, val_x, val_y
        gc.collect()
        
    print('Full AUC score %.6f' % roc_auc_score(y_, oof_preds)) 
    
    test_['isDefault'] = sub_preds

    return oof_preds, test_[['loan_id', 'isDefault']], feature_importance_df
    
def display_importances(feature_importance_df_):
    # Plot feature importances
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(
        by="importance", ascending=False)[:50].index
    
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    
    plt.figure(figsize=(8,10))
    sns.barplot(x="importance", y="feature", 
                data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances.png')

## 特征工程 

In [5]:
work_year_dict = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10,
}

train_data['work_year'] = train_data['work_year'].map(work_year_dict)
test_public['work_year'] = test_public['work_year'].map(work_year_dict)
train_data['work_year'] = train_data['work_year'].fillna(-1)
test_public['work_year'] = test_public['work_year'].fillna(-1)
train_internet['work_year'] = train_internet['work_year'].map(work_year_dict)
train_internet['work_year'] = train_internet['work_year'].fillna(-1)

In [6]:
class_dict = {
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6,
    'G': 7,
}

train_data['class'] = train_data['class'].map(class_dict)
test_public['class'] = test_public['class'].map(class_dict)
train_internet['class'] = train_internet['class'].map(class_dict)

In [7]:
train_data['issue_date'] = pd.to_datetime(train_data['issue_date'])
test_public['issue_date'] = pd.to_datetime(test_public['issue_date'])
train_internet['issue_date'] = pd.to_datetime(train_internet['issue_date'])

train_data['issue_date_month'] = train_data['issue_date'].dt.month
test_public['issue_date_month'] = train_data['issue_date'].dt.month
train_internet['issue_date_month'] = train_internet['issue_date'].dt.month

train_data['issue_date_dayofweek'] = train_data['issue_date'].dt.dayofweek
test_public['issue_date_dayofweek'] = train_data['issue_date'].dt.dayofweek
train_internet['issue_date_dayofweek'] = train_internet['issue_date'].dt.dayofweek


In [8]:
for data in [train_data, test_public, train_internet]: 
    data['earlies_credit_mon'] = pd.to_datetime(data['earlies_credit_mon'],format='%Y-%m-%d',errors='coerce')
    

In [9]:
train_data['earlies_credit_mon'] = pd.to_datetime(train_data['earlies_credit_mon'])
train_data['earlies_credit_Mon'] = train_data['earlies_credit_mon'].dt.month
train_data['earlies_credit_Year'] = train_data['earlies_credit_mon'].dt.year

In [10]:
test_public['earlies_credit_mon'] = pd.to_datetime(test_public['earlies_credit_mon'])
test_public['earlies_credit_Mon'] = test_public['earlies_credit_mon'].dt.month
test_public['earlies_credit_Year'] = test_public['earlies_credit_mon'].dt.year

train_internet['earlies_credit_mon'] = pd.to_datetime(train_internet['earlies_credit_mon'])
train_internet['earlies_credit_Mon'] = train_internet['earlies_credit_mon'].dt.month
train_internet['earlies_credit_Year'] = train_internet['earlies_credit_mon'].dt.year

In [11]:
col_to_drop = ['issue_date', 'earlies_credit_mon', 'issue_date_month', 'earlies_credit_Mon','earlies_credit_Year']
train_data = train_data.drop(col_to_drop, axis=1)
test_public = test_public.drop(col_to_drop, axis=1)
train_internet = train_internet.drop(col_to_drop, axis=1 )

In [12]:
from sklearn.preprocessing import LabelEncoder

cat_cols = ['employer_type', 'industry']
for col in cat_cols:
    lbl = LabelEncoder().fit(train_data[col])
    train_data[col] = lbl.transform(train_data[col])
    test_public[col] = lbl.transform(test_public[col])
    train_internet[col] = lbl.transform(train_internet[col])

In [13]:
tr_cols = set(train_data.columns)
same_col = list(tr_cols.intersection(set(train_internet.columns)))
train_inteSame = train_internet[same_col].copy()

Inte_add_cos = list(tr_cols.difference(set(same_col)))
for col in Inte_add_cos:
    train_inteSame[col] = np.nan

In [14]:
y = train_data['isDefault']
folds = KFold(n_splits=20, shuffle=True, random_state=546789)
oof_preds, IntePre, importances = train_model(train_data, train_inteSame, y, folds)

IntePre['isDef'] = train_internet['is_default']
from sklearn.metrics import roc_auc_score
roc_auc_score(IntePre['isDef'],IntePre.isDefault)

Training until validation scores don't improve for 50 rounds
[100]	training's auc: 0.947444	training's binary_logloss: 0.187846	valid_1's auc: 0.910928	valid_1's binary_logloss: 0.21828
[200]	training's auc: 0.97465	training's binary_logloss: 0.15403	valid_1's auc: 0.914014	valid_1's binary_logloss: 0.214375
Early stopping, best iteration is:
[193]	training's auc: 0.97324	training's binary_logloss: 0.156032	valid_1's auc: 0.914538	valid_1's binary_logloss: 0.214018
Fold  1 AUC : 0.914538
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[36]	training's auc: 0.921883	training's binary_logloss: 0.218015	valid_1's auc: 0.935007	valid_1's binary_logloss: 0.207101
Fold  2 AUC : 0.935007
Training until validation scores don't improve for 50 rounds
[100]	training's auc: 0.948072	training's binary_logloss: 0.187377	valid_1's auc: 0.901227	valid_1's binary_logloss: 0.215495
Early stopping, best iteration is:
[68]	training's auc: 0.93622	training's b

E:\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.733116075476391

In [15]:
InteId = IntePre.loc[IntePre.isDefault<0.5, 'loan_id'].tolist()

train_data['dataSourse'] = 1
test_public['dataSourse'] = 1
train_inteSame['dataSourse'] = 0
train_inteSame['isDefault'] = train_internet['is_default']
use_te = train_inteSame[train_inteSame.loan_id.isin( InteId )].copy()
data = pd.concat([ train_data,test_public,use_te]).reset_index(drop=True)

In [17]:
del data
del train_data,test_public


y = train['isDefault']
folds = KFold(n_splits=20, shuffle=True, random_state=546789)
oof_preds, test_preds, importances = train_model(train, test, y, folds)
test_preds.rename({'loan_id': 'id'}, axis=1)[['id', 'isDefault']].to_csv('./submit/test2.csv', index=False)

Training until validation scores don't improve for 50 rounds
[100]	training's auc: 0.790369	training's binary_logloss: 0.382353	valid_1's auc: 0.783492	valid_1's binary_logloss: 0.386024
[200]	training's auc: 0.796347	training's binary_logloss: 0.378213	valid_1's auc: 0.786886	valid_1's binary_logloss: 0.383757
[300]	training's auc: 0.800032	training's binary_logloss: 0.375723	valid_1's auc: 0.78778	valid_1's binary_logloss: 0.383139
[400]	training's auc: 0.802915	training's binary_logloss: 0.373787	valid_1's auc: 0.788201	valid_1's binary_logloss: 0.382855
[500]	training's auc: 0.805842	training's binary_logloss: 0.371834	valid_1's auc: 0.788583	valid_1's binary_logloss: 0.382613
Early stopping, best iteration is:
[489]	training's auc: 0.805511	training's binary_logloss: 0.372053	valid_1's auc: 0.788628	valid_1's binary_logloss: 0.382572
Fold  1 AUC : 0.788628
Training until validation scores don't improve for 50 rounds
[100]	training's auc: 0.79013	training's binary_logloss: 0.382403

Fold  8 AUC : 0.796111
Training until validation scores don't improve for 50 rounds
[100]	training's auc: 0.790124	training's binary_logloss: 0.382679	valid_1's auc: 0.786621	valid_1's binary_logloss: 0.381824
[200]	training's auc: 0.796007	training's binary_logloss: 0.378586	valid_1's auc: 0.789436	valid_1's binary_logloss: 0.379743
[300]	training's auc: 0.799818	training's binary_logloss: 0.376028	valid_1's auc: 0.790527	valid_1's binary_logloss: 0.378958
[400]	training's auc: 0.802796	training's binary_logloss: 0.37401	valid_1's auc: 0.791033	valid_1's binary_logloss: 0.378594
[500]	training's auc: 0.805624	training's binary_logloss: 0.372126	valid_1's auc: 0.791432	valid_1's binary_logloss: 0.378315
[600]	training's auc: 0.808305	training's binary_logloss: 0.370326	valid_1's auc: 0.791582	valid_1's binary_logloss: 0.378205
[700]	training's auc: 0.810637	training's binary_logloss: 0.36874	valid_1's auc: 0.791738	valid_1's binary_logloss: 0.378148
Early stopping, best iteration is:
[

[200]	training's auc: 0.795791	training's binary_logloss: 0.37851	valid_1's auc: 0.797191	valid_1's binary_logloss: 0.378709
[300]	training's auc: 0.799361	training's binary_logloss: 0.376117	valid_1's auc: 0.798271	valid_1's binary_logloss: 0.37784
[400]	training's auc: 0.802215	training's binary_logloss: 0.374199	valid_1's auc: 0.799	valid_1's binary_logloss: 0.377348
[500]	training's auc: 0.805079	training's binary_logloss: 0.372276	valid_1's auc: 0.799366	valid_1's binary_logloss: 0.377026
[600]	training's auc: 0.807726	training's binary_logloss: 0.370516	valid_1's auc: 0.799511	valid_1's binary_logloss: 0.376864
Early stopping, best iteration is:
[582]	training's auc: 0.807305	training's binary_logloss: 0.37081	valid_1's auc: 0.799569	valid_1's binary_logloss: 0.376834
Fold 16 AUC : 0.799569
Training until validation scores don't improve for 50 rounds
[100]	training's auc: 0.790126	training's binary_logloss: 0.382532	valid_1's auc: 0.787397	valid_1's binary_logloss: 0.383899
[200]

In [18]:
test_preds

,loan_id,isDefault
14472,1000575,0.002394
14473,1028125,0.030103
14474,1010694,0.000849
14475,1026712,0.002198
14476,1002895,0.002462
...,...,...
19467,1008856,0.342985
19468,1016651,0.005640
19469,1024140,0.000628
19470,1014316,0.002904
